# Week 13

Effects of activation functions

In [ ]:
!wget -q https://github.com/PSAM-5020-2025F-A/5020-utils/raw/main/src/data_utils.py

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

from torch import nn, Tensor
from torch.optim import SGD

from data_utils import object_from_json_url, regression_error

# House Price Regression

In [ ]:
# Load Data
HOUSES_FILE = "https://raw.githubusercontent.com/PSAM-5020-2025F-A/5020-utils/main/datasets/json/LA_housing.json"
houses_info = object_from_json_url(HOUSES_FILE)

# DataFrame it
houses_raw_df = pd.DataFrame.from_records(houses_info)

# Scale it
# Note: (technically we should split, then scale, but this saves a few lines of code)
house_scaler = StandardScaler().set_output(transform="pandas")
houses_df = house_scaler.fit_transform(houses_raw_df)

# Ignore outliers
houses_df = houses_df[(houses_df["longitude"] > -2) & (houses_df["longitude"] < 2)]
houses_df = houses_df[(houses_df["value"] > -2) & (houses_df["value"] < 2.5)]

# Train/Test split
houses_train_df, houses_test_df = train_test_split(houses_df, test_size=0.2, random_state=1010)

## Classic Linear Regression

Let's set up a `LinearRegression()` model to predict house prices.

In [ ]:
train_feat = houses_train_df[["longitude"]].values
train_out = houses_train_df[["value"]].values

test_feat = houses_test_df[["longitude"]].values
test_out = houses_test_df[["value"]].values

model = LinearRegression().fit(train_feat, train_out)

train_pred = model.predict(train_feat)
test_pred = model.predict(test_feat)

print("train error", regression_error(train_out, train_pred))
print("test error", regression_error(test_out, test_pred))

In [ ]:
plt.scatter(train_feat[:,0], train_out[:,0], s=3, alpha=0.15, c="C0")
plt.scatter(train_feat[:,0], train_pred[:,0], s=3, alpha=0.75, c="C1")

plt.title("Linear Regression")
plt.xlabel("longitude")
plt.ylabel("value")

plt.show()

### Polynomial Features

In [ ]:
train_feat = houses_train_df[["longitude"]].values
train_out = houses_train_df[["value"]].values

test_feat = houses_test_df[["longitude"]].values
test_out = houses_test_df[["value"]].values

poly = PolynomialFeatures(degree=8).set_output(transform="pandas")

train_poly_feat = poly.fit_transform(train_feat).values

test_poly_feat = poly.transform(test_feat).values

model = LinearRegression().fit(train_poly_feat, train_out)

train_poly_pred = model.predict(train_poly_feat)
test_poly_pred = model.predict(test_poly_feat)

print("train error", regression_error(train_out, train_poly_pred))
print("test error", regression_error(test_out, test_poly_pred))

In [ ]:
plt.scatter(train_feat[:,0], train_out[:,0], s=3, alpha=0.15, c="C0")
plt.scatter(train_feat[:,0], train_poly_pred[:,0], s=3, alpha=0.75, c="C1")

plt.title("Polynomial Linear Regression")
plt.xlabel("longitude")
plt.ylabel("value")

plt.show()

## Neural Network Linear Regression

In [ ]:
x_train = Tensor(train_feat)
y_train = Tensor(train_out)

x_test = Tensor(test_feat)
y_test = Tensor(test_out)

### Single-layer model

In [ ]:
model = nn.Linear(x_train.shape[1], 1)
optim = SGD(model.parameters(), lr=0.05, momentum=0.9)
loss_fn = nn.MSELoss()

In [ ]:
for c in range(32):
  optim.zero_grad()
  y_pred = model(x_train)
  loss = loss_fn(y_pred, y_train)
  loss.backward()
  optim.step()

  if c % 231 == 0:
    print(c, loss.item())

# Evaluate
y_train_pred = model(x_train).tolist()
y_test_pred = model(x_test).tolist()
y_train_pred_flat = [y[0] for y in y_train_pred]

print("train error", regression_error(y_train, y_train_pred))
print("test error", regression_error(y_test, y_test_pred))

In [ ]:
plt.scatter(train_feat[:,0], train_out[:,0], s=3, alpha=0.15, c="C0")
plt.scatter(train_feat[:,0], y_train_pred_flat, s=3, alpha=0.75, c="C1")

plt.title("Single Layer NN")
plt.xlabel("longitude")
plt.ylabel("value")

plt.show()

## Add Layer

In [ ]:
model = nn.Sequential(
  nn.Linear(x_train.shape[1], x_train.shape[1]),
  nn.ReLU(),
  nn.Linear(x_train.shape[1], 1),
)

optim = SGD(model.parameters(), lr=0.01, momentum=0.9)
loss_fn = nn.MSELoss()

### Train

In [ ]:
for c in range(256):
  optim.zero_grad()
  y_pred = model(x_train)
  loss = loss_fn(y_pred, y_train)
  loss.backward()
  optim.step()

  if c % 255 == 0:
    print(c, loss.item())

# Evaluate
y_train_relu_pred = model(x_train).tolist()
y_test_relu_pred = model(x_test).tolist()
y_train_relu_pred_flat = [y[0] for y in y_train_relu_pred]

print("train error", regression_error(y_train, y_train_relu_pred))
print("test error", regression_error(y_test, y_test_relu_pred))

In [ ]:
plt.scatter(train_feat[:,0], train_out[:,0], s=3, alpha=0.15, c="C0")
plt.scatter(train_feat[:,0], y_train_relu_pred_flat, s=3, alpha=0.75, c="C1")

plt.title("Two Layers (Narrow)")
plt.xlabel("longitude")
plt.ylabel("value")

plt.show()

### Add Capacity

In [ ]:
model = nn.Sequential(
  nn.Linear(x_train.shape[1], 32*x_train.shape[1]),
  nn.ReLU(),
  nn.Linear(32*x_train.shape[1], 1),
)

optim = SGD(model.parameters(), lr=0.01, momentum=0.9)
loss_fn = nn.MSELoss()

### Train

In [ ]:
for c in range(1024):
  optim.zero_grad()
  y_pred = model(x_train)
  loss = loss_fn(y_pred, y_train)
  loss.backward()
  optim.step()

  if c % 1023 == 0:
    print(c, loss.item())

# Evaluate
y_train_cap_pred = model(x_train).tolist()
y_test_cap_pred = model(x_test).tolist()
y_train_cap_pred_flat = [y[0] for y in y_train_cap_pred]

print("train error", regression_error(y_train, y_train_cap_pred))
print("test error", regression_error(y_test, y_test_cap_pred))

In [ ]:
plt.scatter(train_feat[:,0], train_out[:,0], s=3, alpha=0.15, c="C0")
plt.scatter(train_feat[:,0], y_train_cap_pred_flat, s=3, alpha=0.75, c="C1")

plt.xlim((-2, 2))
plt.ylim((-2, 2.5))

plt.title("Two Layers (Wide)")
plt.xlabel("longitude")
plt.ylabel("value")

plt.show()